# Module 3

### Query 3.1
Import the file 'gold.csv' (you will find this in the intro section to download or in '/Data/gold.csv' if you are using the jupyter notebook), which contains the data of the last 2 years price action of Indian (MCX) gold standard. Explore the dataframe. You'd see 2 unique columns - 'Pred' and 'new'. One of the 2 columns is a linear combination of the OHLC prices with varying coefficients while the other is a polynomial function of the same inputs. Also, one of the 2 columns is partially filled.
* Using linear regression, find the coefficients of the inputs and using the same trained model, complete the entire column.
* Also, try to fit the other column as well using a new linear regression model. Check if the predictions are accurate. Mention which column is a linear function and which is polynomial.
(Hint: Plotting a histogram & distplot helps in recognizing the discrepencies in prediction, if any.)

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
%matplotlib notebook

In [2]:
dataset = pd.read_csv("GOLD.csv")

In [3]:
dataset.Date.astype('datetime64')
dataset.set_index('Date',inplace = True)
dataset.head()

,Price,Open,High,Low,Vol.,Change %,Pred,new
Date,,,,,,,,
"May 04, 2017",28060,28400,28482,28025,0.08K,-1.79%,738.0,117.570740
"May 05, 2017",28184,28136,28382,28135,0.06K,0.44%,-146.0,295.430176
"May 08, 2017",28119,28145,28255,28097,7.85K,-0.23%,30.0,132.123714
"May 09, 2017",27981,28125,28192,27947,10.10K,-0.49%,357.0,101.298064
"May 10, 2017",28007,28060,28146,27981,9.28K,0.09%,124.0,112.153318


In [4]:
#For Pred Column
X_Pred = dataset.iloc[:,:4]
y_Pred = dataset.iloc[:,-2]
y_Pred = y_Pred.dropna()
len_pred = len(y_Pred)

In [5]:
val = int(len_pred/2)
X_train = X_Pred[:val]
X_test = X_Pred[val : len_pred]

y_train = y_Pred[:val]
y_test = y_Pred[val:len_pred]

X_val_pred = X_Pred[len_pred:]

In [6]:
from sklearn.linear_model import LinearRegression
model_pred = LinearRegression()
model_pred.fit(X_train,y_train)
y_calc = model_pred.predict(X_test)

In [7]:
from sklearn.metrics import r2_score
r2_score(y_test,y_calc)

1.0

In [8]:
#For new column
X_new = dataset.iloc[:,:4]
y_new = dataset.iloc[:,-1]
len_new = len(y_new)

In [9]:
val = int(len_new/2)
X_train = X_new[:val]
X_test = X_new[val : len_new]

y_train = y_new[:val]
y_test = y_new[val:len_new]

X_val_new = X_new[len_new:]

In [10]:
model_new = LinearRegression()
model_new.fit(X_train,y_train)
y_pred_new = model_new.predict(X_test)

In [11]:
r2_score(y_test,y_pred_new)

0.9999830622110687

#### Since r2 score of the pred column is greater than the score of new column,therefore it is linearly dependent with OHLC and new is polynomial function of OHLC

In [12]:
y_pred = model_pred.predict(X_val_pred)
dataset['Pred'][len_pred:] = y_pred

CAPM CAPM Analysis and Beta Calculation using regression - <br>
CAPM(Capital Asset Pricing Model) attempts to price securities by examining the relationship that exists between expected returns and risk. 
Read more about CAPM. (Investopedia CAPM reference) 
The Beta of an asset is a measure of the sensitivity of its returns relative to a market benchmark (usually a market index). How sensitive/insensitive is the returns of an asset to the overall market returns (usually a market index like S&P 500 index). What happens when the market jumps, does the returns of the asset jump accordingly or jump somehow? 
Read more about Beta (Investopedia Beta reference)


### Query3.2
Import the stock of your choosing AND the Nifty index. 
Using linear regression (OLS), calculate -
* The daily Beta value for the past 3 months. (Daily= Daily returns)
* The monthly Beta value. (Monthly= Monthly returns)<p>
Refrain from using the (covariance(x,y)/variance(x)) formula.
Attempt the question using regression.(Regression Reference) <br>
Were the Beta values more or less than 1 ? What if it was negative ? <br>
Discuss. Include a brief writeup in the bottom of your jupyter notebook with your inferences from the Beta values and regression results


In [13]:
Nifty_df = pd.read_csv('Nifty50/Nifty50.csv')
NCC_df = pd.read_csv('Small_Cap/NCC.csv')

NCC_df = NCC_df[NCC_df.Series == 'EQ']

NCC_df['Date'] = NCC_df['Date'].astype('datetime64')
Nifty_df['Date'] = Nifty_df['Date'].astype('datetime64')

NCC_df.set_index('Date',inplace = True)
Nifty_df.set_index('Date',inplace = True)

NCC_df['Day_Precentage_Change'] = NCC_df['Close Price'].pct_change()
Nifty_df['Day_Precentage_Change'] = Nifty_df['Close'].pct_change()

NCC_df.dropna(inplace = True)
Nifty_df.dropna(inplace = True)

In [14]:
import statsmodels.api as sm

In [15]:
Daily_return_df = pd.concat([Nifty_df.iloc[-90:,-1:],NCC_df.iloc[-90:,-1:]],axis = 1)
Daily_return_df.columns = ['Nifty','NCC']

X = Daily_return_df['Nifty']
y = Daily_return_df['NCC']

X1 = sm.add_constant(X)

model_beta = sm.OLS(y,X1)
results = model_beta.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    NCC   R-squared:                       0.122
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     12.21
Date:                Sun, 14 Jul 2019   Prob (F-statistic):           0.000748
Time:                        19:15:37   Log-Likelihood:                 213.22
No. Observations:                  90   AIC:                            -422.4
Df Residuals:                      88   BIC:                            -417.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0003      0.002      0.144      0.8

#### The value of beta for last 3 months on daily returns comes out to be 1.2350 which is greater than 1, this means that the stock NCC more volatile than Nifty on a daily basis.

In [16]:
monthly_return_df = pd.concat([Nifty_df['Close'].resample("M").mean(),NCC_df['Close Price'].resample("M").mean()]
                              ,axis = 1)
monthly_return_df.columns = ['Nifty','NCC']

monthly_return_df = monthly_return_df.pct_change()
monthly_return_df.dropna(inplace = True)

X = monthly_return_df['Nifty']
y = monthly_return_df['NCC']

X1 = sm.add_constant(X)

model_beta_month = sm.OLS(y,X1)
results_month = model_beta_month.fit()
print(results_month.summary())

                            OLS Regression Results                            
Dep. Variable:                    NCC   R-squared:                       0.331
Model:                            OLS   Adj. R-squared:                  0.301
Method:                 Least Squares   F-statistic:                     10.91
Date:                Sun, 14 Jul 2019   Prob (F-statistic):            0.00324
Time:                        19:15:39   Log-Likelihood:                 23.801
No. Observations:                  24   AIC:                            -43.60
Df Residuals:                      22   BIC:                            -41.25
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0121      0.020     -0.604      0.5

#### The value of beta for monthly return comes out to be 2.3394 which is greater than 1, this means that the stock NCC has an has very high volatility than Nifty on monthly basis.

A stock with a beta of:
* 0 indicates their is no correlation between chosen benchmark and itself
* 1 indicates a stock has the same volatility as the market
* more than 1 indicates a stock that’s more volatile than its benchmark
* less than 1 is less volatile than the benchmark
* less than 0 i.e negative beta indicates inverse relation